# Blurred Ball Cover

Data Source: https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

# Import Modules and Packages

In [36]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
from collections import namedtuple
import numpy.linalg as npl
from sets import Set

# Declare Data Structures

In [37]:
Ball = namedtuple("Ball", ["center", "radius"])
BlurredBall = namedtuple("BlurredBall", ["k", "MEB"])

# Methods to Generate and Read Data

In [38]:
def generate_normal_data(num_of_points, num_of_dimensions):
    mean = random.sample(np.array(range(100)), num_of_dimensions)
    scalar_variances = 10.0 * np.ones(num_of_dimensions)
    covariance = np.diag(scalar_variances)
    points = np.random.multivariate_normal(mean, covariance, num_of_points)
    return points

def generate_multivariate_data(num_of_points, num_of_dimensions):
    probabilities = [float(1.0 / num_of_dimensions)] * num_of_dimensions
    points = np.random.multinomial(20, probabilities, size = num_of_points)
    return points

def read_data(data_file):
    data = []
    with open(data_file) as data_reader:
        lines = data_reader.readlines()
        for line in lines:
            values = line.split(",")
            #values = values[1:]
            row = [float(value) for value in values]
            data.append(row)
        data = np.array(data)
        print(data.shape)
        return data, data.shape[0], data.shape[1]


# Helper Functions for Computing Clarkson's (2/epsilon)-coreset and MEB 

In [39]:
def inside_ball_eps(blurred_ball, point, epsilon):
    #print "inside_ball"
    center, radius = blurred_ball.MEB
    distance = npl.norm(point - center)
    if distance <= (1 + epsilon) * radius:
        return True
    else:
        return False

In [40]:
def approx_meb_light(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0
    num_iterations = int(1.0 / (epsilon * epsilon))
    distances = {}
    prev_radius = -10000.0
    coreset = [points[0]]
    for i in range(1, num_iterations):
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]

        center = center + (1.0 / i) * (furthest_point - center)   
        if abs(furthest_distance - prev_radius) < 0.01:
            break
        prev_radius = furthest_distance
            
    radius = npl.norm(furthest_point - center)
    meb = Ball(center, radius)
    return meb

In [41]:
def epsilon_coreset(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0

    num_iterations = int(2.0 / (epsilon))
    print("Total Iterations: " + repr(num_iterations))
    distances = {}
    coreset_indices = Set()

    prev_radius = -10000.0
    
    coreset = [points[0]]
    for i in range(1, num_iterations):
        #print("Iteration: ", i, " radius: ", prev_radius)
        # Find furthest point
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        # Calculated furthest point    
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]
        if furthest_point_index in coreset_indices:
            continue
        coreset_indices.add(furthest_point_index)
        coreset.append(furthest_point)
        coreset_meb = approx_meb_light(coreset, 0.001)
        center = coreset_meb.center
        #print center, coreset_meb.radius
        
    return coreset

# Clarkson's 2/epsilon coreset and MEB Algorithm

In [42]:
def approx_meb(points, epsilon):
    coreset = epsilon_coreset(points, epsilon)
    meb = approx_meb_light(coreset, epsilon)
    return BlurredBall(coreset, meb)
    

# Blurred Ball Cover Algorithm's Helper Method

In [43]:
def update(blurred_balls, A, epsilon):
    print "update"
    K = []
    for blurred_ball in blurred_balls:
        K.extend(blurred_ball.k)


    outer_loop_flag = False    
    for point in A:
        if outer_loop_flag == True:
            break
        for blurred_ball in blurred_balls:
            if inside_ball_eps(blurred_ball, point, epsilon) == False:
                outer_loop_flag = True
                break
                
    if outer_loop_flag == False:
        #print("This point is covered")
        return blurred_balls
    #else:
        #print("This point is not covered")
        
    K_union_A = list(K)
    K_union_A.extend(A)
    blurred_ball_new = approx_meb(K_union_A, epsilon / 3.0)
    
    discardables = []
    for blurred_ball in blurred_balls:
        lhs = blurred_ball.MEB.radius
        rhs = epsilon * blurred_ball_new.MEB.radius / 4.0
        #print("lhs <> rhs: " + repr(lhs) + " <> " + repr(rhs))
        if blurred_ball.MEB.radius <= (epsilon * blurred_ball_new.MEB.radius / 4.0):
            print "Found a discardable"
            discardables.append(blurred_ball)
    
    blurred_balls = [bb for bb in blurred_balls if bb not in np.array(discardables)]
    
    blurred_balls.append(blurred_ball_new)
    
    return blurred_balls
        

# Blurred Ball Cover Algorithm

In [44]:
def agarwal(points, num_of_points, num_of_dimensions):
    epsilon = 0.1
    batch_size = 100
    blurred_balls = []
    initial_points = points[0:num_of_dimensions, :]
    blurred_ball_init = approx_meb(initial_points, epsilon / 3.0)
    blurred_balls = [blurred_ball_init]

    for i in range(num_of_dimensions, num_of_points, batch_size):
        print "Iteration: " + repr(i)
        sid = i
        fid = min(i + batch_size, num_of_points)
        A = points[sid : fid, :]
        blurred_balls = update(blurred_balls, A, epsilon)
        
    return blurred_balls
  

# Read Data from File

In [47]:
points, num_of_points, num_of_dimensions = read_data("Epileptic.csv")
print num_of_points, num_of_dimensions

(11500, 178)
11500 178


# Run Blurred Ball Cover Algorithm

In [48]:
blurred_balls = agarwal(points, num_of_points, num_of_dimensions)
#draw_blurred_balls(blurred_balls)

Total Iterations: 60
Iteration: 178
update
Total Iterations: 60


/Library/Python/2.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Iteration: 278
update
Total Iterations: 60
Iteration: 378
update
Total Iterations: 60
Iteration: 478
update
Total Iterations: 60
Iteration: 578
update
Total Iterations: 60
Iteration: 678
update
Total Iterations: 60
Iteration: 778
update
Total Iterations: 60
Iteration: 878
update
Iteration: 978
update
Total Iterations: 60
Iteration: 1078
update
Total Iterations: 60
Iteration: 1178
update
Total Iterations: 60
Iteration: 1278
update
Total Iterations: 60
Iteration: 1378
update
Total Iterations: 60
Iteration: 1478
update
Total Iterations: 60
Iteration: 1578
update
Total Iterations: 60
Iteration: 1678
update
Total Iterations: 60
Iteration: 1778
update
Total Iterations: 60
Iteration: 1878
update
Total Iterations: 60
Iteration: 1978
update
Total Iterations: 60
Iteration: 2078
update
Total Iterations: 60
Iteration: 2178
update
Total Iterations: 60
Iteration: 2278
update
Total Iterations: 60
Iteration: 2378
update
Total Iterations: 60
Iteration: 2478
update
Total Iterations: 60
Iteration: 2578
u

# Inspect Blurred Ball Cover's Output

In [49]:
for i in range(len(blurred_balls)):
    print i, len(blurred_balls[i].k), blurred_balls[i].MEB.radius

0 8 6814.31505862
1 12 7180.83611459
2 11 7504.71183012
3 13 7852.69461646
4 13 7846.30685041
5 13 7933.89407737
6 14 8052.01127468
7 14 8021.98328456
8 14 8021.98328456
9 14 8021.98328456
10 13 8097.34433351
11 13 8236.36281389
12 13 8236.36281389
13 13 8236.36281389
14 13 8236.36281389
15 13 8236.36281389
16 13 8236.36281389
17 12 8337.53268194
18 12 8337.53268194
19 12 8337.53268194
20 15 8370.43222591
21 15 8407.59409555
22 13 8378.97297809
23 14 8469.42979432
24 14 8469.42979432
25 14 8469.42979432
26 14 8469.42979432
27 15 8613.48065816
28 16 8648.28189574
29 15 8694.59267368
30 14 8699.37839673
31 14 8699.37839673
32 15 8697.2243036
33 14 8711.54827456
34 14 8711.54827456
35 12 8844.98072592
36 12 9176.36824667
37 13 9149.14438024
38 13 9149.14438024
39 14 9194.31507039
40 14 9194.31507039
41 14 9194.31507039
42 14 9194.31507039
43 14 9194.31507039
44 14 9194.31507039
45 14 9194.31507039
46 14 9194.31507039
47 14 9194.31507039
48 14 9194.31507039
49 14 9194.31507039
50 14 9194.3

# Write the Blurred Balls into a File

In [50]:
N = len(blurred_balls)
D = num_of_dimensions

with open("eplileptic_balls.txt", "w") as outFile:
    outFile.write(repr(N) + " " + repr(D) + "\n")
    for ball in blurred_balls:
        print ball.MEB.center, ball.MEB.radius
        line = ""
        for d in range(D):
            line += repr(ball.MEB.center[d]) + " "
        line += repr(ball.MEB.radius) + "\n"
        outFile.write(line)
        #print line

[  3.54874251e+02   4.15497006e+02   3.42532934e+02   2.27485030e+02
   5.52155689e+01  -7.87544910e+01  -1.14137725e+02  -1.05257485e+02
  -3.75538922e+01   4.14760479e+01   8.66167665e+01   2.70598802e+01
  -1.88380240e+02  -3.43943114e+02  -4.13508982e+02  -2.32224551e+02
  -1.24880240e+02  -5.56407186e+01  -9.39940120e+01  -1.38230539e+02
  -1.44931138e+02  -1.41473054e+02  -8.63532934e+01   1.49700599e-02
   1.06470060e+02   1.03652695e+02   1.04745509e+02  -1.28173653e+01
  -6.28982036e+01  -6.88682635e+01  -4.83443114e+01   5.50479042e+01
   7.58532934e+01   1.02263473e+02   1.18185629e+02   1.17167665e+02
   9.64431138e+01   4.45838323e+01  -2.29041916e+01  -2.10179641e+00
   8.06287425e+01   2.35865269e+02   3.60410180e+02   3.99688623e+02
   3.49053892e+02   2.77946108e+02   1.51224551e+02   1.62694611e+01
  -1.18281437e+02  -2.11805389e+02  -2.09520958e+02  -1.17826347e+02
   2.59550898e+01   1.36730539e+02   1.91500000e+02   1.64577844e+02
   8.57035928e+01  -2.62634731e+01

    8.7489301   -57.40370899 -119.04850214] 8469.42979432
[ -69.36363636    4.42857143   68.50649351   89.75324675   84.11688312
   44.24675325    3.27272727  -23.81818182  -13.54545455   20.36363636
   42.27272727    7.53246753  -51.51948052  -62.64935065  -60.72727273
  -17.24675325  133.37662338  288.02597403  390.35064935  426.83116883
  355.12987013  233.05194805   95.31168831   10.4025974   -78.35064935
 -135.35064935  -82.7012987   -30.45454545  -25.06493506  -38.84415584
  -50.74025974  -22.87012987   35.48051948   79.71428571   81.61038961
    8.12987013 -109.83116883 -221.49350649 -258.28571429 -223.92207792
 -154.36363636   -1.64935065   51.03896104  -52.16883117 -249.35064935
 -282.06493506 -208.64935065 -131.45454545  -74.01298701  -48.72727273
  -64.15584416 -109.01298701 -131.02597403  -70.03896104  -38.35064935
  -28.74025974  -32.49350649  -49.02597403  -42.72727273  -16.80519481
   54.33766234  133.61038961  194.12987013  211.4025974   164.68831169
   58.16883117  -42

[ 328.89967638  398.83656958  438.70226537  447.9789644   418.05501618
  349.6763754   271.1407767   214.15533981  190.91747573  203.79288026
  166.06472492    4.0987055  -239.10194175 -366.40938511 -359.83495146
 -282.03398058 -226.25566343 -214.46925566 -223.35113269 -119.47572816
  -23.84466019  138.96440129   72.89644013   -5.71682848  -93.65372168
 -177.88187702 -131.78317152  -53.90453074   47.48543689  128.92718447
  199.08090615  249.98220065  285.16019417  303.14401294  282.08737864
  223.49676375  135.           65.13592233   15.69417476    7.7394822
   20.30906149   97.90614887  114.02912621    2.47249191 -159.87864078
 -159.6763754   -40.83656958   79.72815534  171.64401294  219.51132686
  235.74595469  231.38996764  235.76860841  288.4223301   230.73300971
   48.11812298 -181.50647249 -260.9789644  -261.37864078 -285.56957929
 -306.70226537 -285.27831715 -152.27831715  -64.21521036  -30.37540453
  -36.09061489  -73.30420712 -182.16990291 -236.03721683 -298.94498382
 -269.2

In [51]:
import numpy as np
from sklearn.decomposition import PCA

def get_pca_transformer(points_hd):
    pca = PCA(n_components=2)
    pca.fit(points_hd)
    return pca

def get_blurred_ball_2d(blurred_ball, pca):
    points_hd = blurred_ball.k
    center_hd = blurred_ball.MEB.center
    points_hd.append(center_hd)
    
    points_2d = pca.fit_transform(points_hd)
    num_of_points = len(points_2d)
    center_2d = points_2d[num_of_points - 1, :]
    radius = blurred_ball.MEB.radius
    points_2d = points_2d[:num_of_points - 2, :]
    blurred_ball_2d = BlurredBall(points_2d, Ball(center_2d, radius))
    return blurred_ball_2d

In [52]:
'''
pca = get_pca_transformer(points)
blurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]
print(blurred_balls_2d[0])
draw_blurred_balls(blurred_balls_2d)
'''

'\npca = get_pca_transformer(points)\nblurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]\nprint(blurred_balls_2d[0])\ndraw_blurred_balls(blurred_balls_2d)\n'

# Timothy Chan's Algorithm

In [53]:
Ball = namedtuple("Ball", ["center", "radius"])

def inside_ball(ball, point):
    center, radius = ball
    distance = npl.norm(point - center)
    if distance < radius:
        return True
    else:
        return False

def meb_ball_and_point(ball, p):
    c, r = ball
    pc_scalar = npl.norm(c - p)
    pc_vector = c - p
    radius_unit = pc_vector / pc_scalar
    p_mirror = radius_unit * r + c
    c_prime = (p + p_mirror) / 2.0
    r_prime = npl.norm(p_mirror - p) / 2.0
    meb = Ball(c_prime, r_prime)
    return meb

def create_initial_ball(point):
    return Ball(point, 0.0)

def chan(points):
    count = 0
    center = None
    radius = None
    ball = None
    for point in points:
        if ball == None:
            ball = create_initial_ball(point)
            continue
        if inside_ball(ball, point):
            continue
        else:
            count += 1
            ball = meb_ball_and_point(ball, point)
            center, radius = ball
    return Ball(center, radius)

In [54]:
meb = chan(points)
print(meb)
    

Ball(center=array([ 230.55443039,  239.89686225,  208.88855855,  164.74136275,
        104.38149688,   53.51500988,   27.18375195,   22.78792731,
         45.81689292,   74.83599871,   82.83299359,   25.65333329,
        -88.6780237 , -203.35991028, -262.75591198, -269.22875925,
       -256.38770776, -247.6667294 , -260.08462857, -245.77692483,
       -212.25995619, -141.04514437,  -90.88474786,  -42.23459481,
        -14.00831123,  -15.88662286,   -6.020165  ,   31.15056288,
         85.99816339,  130.0619107 ,  158.19316389,  165.38677055,
        159.62181453,  144.18759939,  116.99481668,   84.21627665,
         47.56392703,   18.52179954,   15.57141244,   43.7033497 ,
         81.86849302,  111.08981593,  132.70958057,  131.71824859,
        118.72339104,  109.8134925 ,   85.80068531,   50.9213118 ,
         23.1402683 ,    6.17985647,   25.56999504,   78.58037494,
        136.28006676,  171.86655419,  168.12752082,  115.99662437,
         26.16087092,  -57.97753071, -128.75483448

# TODO

## Compute MEB from MEBs

# Extension of Chan's Algorithm?